In [1]:
import script.compile as compiler
from script.model import model_cal, grid_search
import numpy as np
from scipy.stats import norm
import pandas as pd
import random
from script.compile import comp_times_base

In [2]:
data, desc, cols = compiler.data_loader(convert = False, 
                                        desc_names = ['electronegativity','delta_fus H','density'])

In [3]:
model_cal(data, cols, shap = False)

conv


KeyboardInterrupt: 

,Li,Be,B,C,N,F,Na,Mg,Al,Si,...,AN (4),AW (4),atomic radius (4),electronegativity (4),m. p. (4),b. p. (4),delta_fus H (4),density (4),ionization enegy (4),Surface energy (4)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.284885,0.0,...,15.24925,37.284509,0.527859,0.414825,382.169656,821.817254,2.381229,1.93079,1.37567,0.227566
160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.284885,0.0,...,15.24925,37.284509,0.527859,0.414825,382.169656,821.817254,2.381229,1.93079,1.37567,0.227566
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.284885,0.0,...,15.24925,37.284509,0.527859,0.414825,382.169656,821.817254,2.381229,1.93079,1.37567,0.227566


In [14]:
def smac(model, init_x, init_y, roen_func, desc, cols, data, random_state = 1126):
    """
    SMAC　のメインプログラム
    """
    model.fit(np.array(init_x.loc[:, cols['prop2']]), np.array(init_y))
    print(model)
    mu, sigma =  posterior(init_x.loc[:, cols['prop2']] , init_x.loc[:, cols['prop2']], init_y, model)
    ei =  EI(mu, sigma, init_y.max())
    ei = pd.Series(ei, index = init_x.index, name = 'ei')
    make_nei =pd.Series(True, index=init_x.index, name='make_nei')
    next_x = pd.concat([init_x, ei, make_nei], axis =1)
    while next_x['make_nei'].sum() != 0:
        next_x = roen_func(next_x, init_y, model, desc, cols, data)
        print(next_x['make_nei'].sum())
        
    return next_x

def opt_function(x, y ,model, desc, cols, data, random_state =1126):
    """
    各実験ずみの点に対して4回近傍の探索を行い最もEIの高かった組成実験条件を返す
    """
    xx = x.iloc[:,:-2]
    neighbor = pd.DataFrame(index = x.columns)
    for key, row in xx.iterrows():
        nei_cand = []
        #print('ind={0}'.format(key))
        if x.loc[key, 'make_nei'] == True:

            for _ in range(30):
                row_ch = row.copy()
                
                row_ch = SWED_change(row_ch, desc, cols, data)

                nei_cand.append(row_ch)
                
            for _ in range(30):
                row_ch = row.copy()
                
                row_ch = experiment_change(row_ch, cols, data)
                
                nei_cand.append(row_ch)
                
            for _ in range(30):
                row_ch = row.copy()
                
                row_ch = SWED_change(row_ch, desc, cols, data)
                
                row_ch = experiment_change(row_ch, cols, data)
                
                nei_cand.append(row_ch)
                   

            nei_cand = pd.DataFrame(nei_cand, index =np.arange(len(nei_cand))).fillna(0)
            swed = comp_times_base(nei_cand.loc[:,cols['element']],
                                   desc.loc[cols['element']].T,sort=True,times=True)
            swed = pd.DataFrame(swed).iloc[:, :len(cols['swed'])].fillna(0)
            nei_cand.loc[:, cols['swed']] = np.array(swed)
                        
            mu, sigma = model.predict(np.array(nei_cand.loc[:, cols['prop2']]), return_std=True)
            ind = y.values.argmax()
            cur_max = y.iloc[ind]
            ei = EI(mu, sigma, cur_max)
            ind = np.argmax(ei)
            cand = nei_cand.iloc[ind].copy()
            cand['ei'] = ei[ind]
            if x.loc[key, 'ei'] < cand['ei']:
                cand['make_nei'] = True
                neighbor = pd.concat([neighbor, cand], axis = 1)
            else:
                x.loc[key, 'make_nei'] = False
                neighbor = pd.concat([neighbor, x.loc[key,:]], axis = 1)
                
        else:
            neighbor = pd.concat([neighbor, x.loc[key,:]], axis = 1)
                                                    
    print('-----------')
    
    neighbor = neighbor.T
    neighbor.index = x.index
    return neighbor


def SWED_change(row_ch, desc, cols, data):
    row_ch[cols['element']] = 0

    for i in range(0, 3):
        #i-1回目の処理のデータを保存
        row_sub = row_ch
        x_ch =data

        #大きい方からchange_f番目の元素に紐ずくaddtional descriptorを一括での局所的探索
        
        change_col= []
        for s in cols['use_cols']:
            change_col.append(f'{s} ({i + 1})')
        a = row_ch[change_col] - x_ch[change_col].min()
        b = x_ch[change_col].max() - x_ch[change_col].min()
        v = a/b

        v = np.minimum(v, 1.0)
        v = np.maximum(v, 0.0)
        p = np.array([-1])
        count = 0
        while (p <0).any() | (p>1).any():
            p = random.normalvariate(v, 0.05)
            count += 1
            if (count % 1000) == 0:
                p = v
                break

        p = p * b + x_ch[change_col].min()
        row_ch[change_col] = p
        p = np.array(p)
        
        #alpha = x1_x0/x1_2#２次関数の軸の位置
        alpha = desc.apply(lambda u: np.dot(u, p)/np.dot(u, u), axis = 1)
        #εの値が小さい元素名を返す
        epsilon = desc.mul(alpha, axis=0).apply(lambda u: np.dot(u-p, u-p), axis=1).sort_values().index


        for i in range(0, len(epsilon)):
            #軸が100以下をみたし、　組成元素が0ではない元素に変換する
            if (alpha[epsilon[i]] < 100) & (row_ch[epsilon[i]] == 0):
                row_ch[epsilon[i]] = alpha[epsilon[i]]
                break

        #組成の合計が100を超えると処理を抜ける        
        if row_ch[cols['element']].sum() >= 100:
            break
    
    if row_sub[cols['element']].sum() == 0:
        print('enconunter divides zero')

    # 100によりちかい組成を選択肢、標準化
    if  abs(100 -row_ch[cols['element']].sum()) > abs(100-row_sub[cols['element']].sum()):
        row_ch[cols['element']] =  (row_sub[cols['element']] * 100) / row_sub[cols['element']].sum()

    else:
        row_ch[cols['element']] =  (row_ch[cols['element']] * 100) / row_ch[cols['element']].sum()
    
    return row_ch

def experiment_change(row_ch, cols, data):
    x_ch = data

    change_f = random.choice(cols['condition'] + cols['preparation'])
    
    if change_f in cols['condition']:
        a = np.array(row_ch[change_f]) - x_ch[change_f].min()
        b = x_ch[change_f].max() - x_ch[change_f].min()
        if b != 0:
            v = a / b
            p = -1
            while (p < 0) | (p > 1):
                p = random.normalvariate(v, 0.05)

            p = p * b + x_ch[change_f].min()
            row_ch[change_f] = p

    elif change_f in cols['preparation']:
        row_ch[change_f] = 1
        ind = set(cols['preparation']) - {change_f}
        row_ch[ind] = 0
        
    return row_ch

def posterior(x, p_x, p_y, model):
    """
    EIを計算する上でのμならびにσの計算(鈴木氏作成)
    """
    if len(p_x.shape) == 1:
        model.fit(p_x.reshape(-1, 1), p_y)
        mu, sigma = model.predict(x.reshape(-1, 1), return_std = True)
    else:
        model.fit(p_x, p_y)
        mu, sigma = model.predict(x, return_std = True)
    ind = np.where(sigma == 0)
    sigma[ind] = 1e-5
    return mu, sigma

def EI(mu, sigma, cur_max):
    """
    EIの計算(鈴木氏作成)
    """
    Z = (mu - cur_max)/ sigma
    ei = (mu - cur_max) * norm.cdf(Z) + sigma*norm.pdf(Z)
    return ei

In [4]:
data, desc, cols = compiler.data_loader(convert = False, temp = 150)
idx = (data.loc[:,cols['element']] > 0).sum(axis =1) <= 3
data =data[idx]
model = grid_search(data.loc[:, cols['prop2']], data.loc[:, cols['target']])

RMSE 1.705 (sd: 0.194, min:1.176, max:1.835, det:0.994) ... train
RMSE 11.325 (sd: 2.781, min:7.220, max:15.312, det:0.513) ... test


In [ ]:
smac(model, data.loc[:, cols['prop1']], data.loc[:, cols['target']], opt_function,
     desc, cols, data, random_state = 1126)

ExtraTreesRegressor(n_estimators=1000, n_jobs=-1, random_state=1126)


divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true

-----------
160.0


divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide
divide by zero encountered in true_divide


In [6]:
data.loc[:, cols['swed']].describe().to_csv('out/sub.csv')